In [1]:
import os
import re
import pandas as pd


def load_rock_mds_files(folder):
    pattern = re.compile(r"rocks_mds(\d+)\.txt")
    dfs = []
    for fname in os.listdir(folder):
        match = pattern.match(fname)
        if match:
            subject_number = int(match.group(1))
            df = pd.read_csv(
                os.path.join(folder, fname), delim_whitespace=True, engine="python"
            )
            df["Subject Number"] = subject_number
            dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # Empty if no files found

In [2]:
all_data = load_rock_mds_files(
    "data/Rocks-30 Similarity-Judgment Study/individual_subject_data"
)

C:\Users\crasa\AppData\Local\Temp\ipykernel_39992\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_39992\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_39992\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_39992\2038370909.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_39992\2038370909.py:13: FutureWarnin

In [3]:
all_data["RockPair"] = all_data["RockPair"] = all_data.apply(
    lambda row: tuple(sorted([row["Rock1Type"], row["Rock2Type"]])), axis=1
)
all_data

,Rock1Type,Rock2Type,Rating,Subject Number,RockPair
0,20,18,3,10,"(18, 20)"
1,10,14,7,10,"(10, 14)"
2,4,28,2,10,"(4, 28)"
3,6,24,3,10,"(6, 24)"
4,13,30,6,10,"(13, 30)"
...,...,...,...,...,...
30880,5,22,4,9,"(5, 22)"
30881,4,1,6,9,"(1, 4)"
30882,4,7,5,9,"(4, 7)"
30883,25,14,4,9,"(14, 25)"


In [4]:
len(all_data["Subject Number"].unique())

71

In [5]:
all_data.groupby(["RockPair"])["Rating"].mean().reset_index()

,RockPair,Rating
0,"(1, 2)",5.647887
1,"(1, 3)",6.676056
2,"(1, 4)",6.788732
3,"(1, 5)",5.605634
4,"(1, 6)",3.647887
...,...,...
430,"(27, 29)",2.450704
431,"(27, 30)",3.169014
432,"(28, 29)",1.859155
433,"(28, 30)",2.887324


In [6]:
df = all_data

results = []

for subject in df["Subject Number"].unique():
    # Data for this subject
    subj_df = df[df["Subject Number"] == subject]
    # Data for all other subjects
    others_df = df[df["Subject Number"] != subject]
    # Mean ratings from others, grouped by Rock1Type and Rock2Type
    mean_ratings = others_df.groupby(["RockPair"])["Rating"].mean().reset_index()
    mean_ratings = mean_ratings.rename(columns={"Rating": "OthersMeanRating"})
    # Merge with this subject's ratings
    merged = pd.merge(subj_df, mean_ratings, on=["RockPair"], how="inner")
    # Compute correlation
    corr = merged["Rating"].corr(merged["OthersMeanRating"])
    results.append({"Subject Number": subject, "Correlation": corr})

result_df = pd.DataFrame(results)
result_df

,Subject Number,Correlation
0,10,0.695675
1,11,0.383672
2,14,0.400213
3,15,0.591911
4,16,0.719669
...,...,...
66,83,0.617614
67,84,0.542280
68,85,0.532101
69,86,0.724805


In [ ]:
result_df.to_csv("output/30_rock_indv_correlations.csv")

In [8]:
result_df["Correlation"].mean()

np.float64(0.5790505496883465)

In [9]:
result_df["Correlation"].std()

np.float64(0.1167175441731196)

In [10]:
import numpy as np

corrs = []
for _ in range(1000):
    subjects = df["Subject Number"].unique()
    np.random.shuffle(subjects)

    # Split into two halves
    half = len(subjects) // 2
    group1 = subjects[:half]
    group2 = subjects[half:]

    # Aggregate average ratings by RockPair for each group
    avg1 = (
        df[df["Subject Number"].isin(group1)]
        .groupby("RockPair")["Rating"]
        .mean()
        .reset_index()
    )
    avg2 = (
        df[df["Subject Number"].isin(group2)]
        .groupby("RockPair")["Rating"]
        .mean()
        .reset_index()
    )

    corr = avg1.merge(avg2, on="RockPair").corr(numeric_only=True).iloc[0, 1]
    corrs.append(float(corr))

In [ ]:
with open("output/30_rock_split_half_correlations.txt", "w") as f:
    for corr in corrs:
        f.write(f"{corr}\n")

In [12]:
np.mean(corrs)

np.float64(0.9487398148160763)

In [13]:
np.std(corrs)

np.float64(0.006312353430008039)

In [33]:
def load_rock_mds_files(folder):
    pattern = re.compile(r"rocks_mds(\d+)\.txt")
    dfs = []
    for fname in os.listdir(folder):
        match = pattern.match(fname)
        if match:
            subject_number = int(match.group(1))
            df = pd.read_csv(
                os.path.join(folder, fname),
                delim_whitespace=True,
                engine="python",
                header=None,
                names=["Rock1Type", "Rock2Type", "Rock1Token", "Rock2Token", "Rating"],
            )
            df["Subject Number"] = subject_number
            dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # Empty if no files found

In [ ]:
df = load_rock_mds_files(
    "data/Rocks-360 Similarity-Judgment Study/individual subject data"
)

C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\3192843496.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\3192843496.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\3192843496.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\3192843496.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\crasa\AppData\Local\Temp\ipykernel_30900\3192843496.py:8: FutureWarning: Th

In [35]:
df["RockPair"] = df["RockPair"] = df.apply(
    lambda row: tuple(sorted([row["Rock1Type"], row["Rock2Type"]])), axis=1
)

In [36]:
df

,Rock1Type,Rock2Type,Rock1Token,Rock2Token,Rating,Subject Number,RockPair
0,14,19,9,7,3,1,"(14, 19)"
1,29,22,6,6,7,1,"(22, 29)"
2,7,6,5,2,4,1,"(6, 7)"
3,22,10,8,6,3,1,"(10, 22)"
4,11,2,7,5,4,1,"(2, 11)"
...,...,...,...,...,...,...,...
117640,24,12,2,3,3,99,"(12, 24)"
117641,3,20,3,2,2,99,"(3, 20)"
117642,1,27,6,10,3,99,"(1, 27)"
117643,24,8,4,5,7,99,"(8, 24)"


In [37]:
len(df["Subject Number"].unique())

253

In [39]:
df.groupby(["RockPair"])["Rating"].mean().reset_index()

,RockPair,Rating
0,"(1, 1)",5.628458
1,"(1, 2)",4.901186
2,"(1, 3)",5.166008
3,"(1, 4)",5.501976
4,"(1, 5)",4.644269
...,...,...
460,"(28, 29)",3.019763
461,"(28, 30)",3.632411
462,"(29, 29)",5.462451
463,"(29, 30)",4.264822


In [40]:
results = []

for subject in df["Subject Number"].unique():
    # Data for this subject
    subj_df = df[df["Subject Number"] == subject]
    # Data for all other subjects
    others_df = df[df["Subject Number"] != subject]
    # Mean ratings from others, grouped by Rock1Type and Rock2Type
    mean_ratings = others_df.groupby(["RockPair"])["Rating"].mean().reset_index()
    mean_ratings = mean_ratings.rename(columns={"Rating": "OthersMeanRating"})
    # Merge with this subject's ratings
    merged = pd.merge(subj_df, mean_ratings, on=["RockPair"], how="inner")
    # Compute correlation
    corr = merged["Rating"].corr(merged["OthersMeanRating"])
    results.append({"Subject Number": subject, "Correlation": corr})

result_df = pd.DataFrame(results)
result_df

,Subject Number,Correlation
0,1,0.449907
1,10,0.569884
2,100,0.227581
3,101,0.358816
4,102,0.560390
...,...,...
248,93,0.300994
249,94,0.378742
250,96,0.286471
251,97,0.342564


In [42]:
result_df["Correlation"].mean()

np.float64(0.38697871837864384)

In [43]:
result_df["Correlation"].std()

np.float64(0.09204815262030379)

In [44]:
corrs = []
for _ in range(1000):
    subjects = df["Subject Number"].unique()
    np.random.shuffle(subjects)

    # Split into two halves
    half = len(subjects) // 2
    group1 = subjects[:half]
    group2 = subjects[half:]

    # Aggregate average ratings by RockPair for each group
    avg1 = (
        df[df["Subject Number"].isin(group1)]
        .groupby("RockPair")["Rating"]
        .mean()
        .reset_index()
    )
    avg2 = (
        df[df["Subject Number"].isin(group2)]
        .groupby("RockPair")["Rating"]
        .mean()
        .reset_index()
    )

    corr = avg1.merge(avg2, on="RockPair").corr(numeric_only=True).iloc[0, 1]
    corrs.append(float(corr))

In [45]:
np.mean(corrs)

np.float64(0.9574575038308327)

In [46]:
np.std(corrs)

np.float64(0.0028415100832826436)